In [1]:
import pandas as pd

In [6]:
# Print all UFC fights for a fighter 

# Define the fighter's name
fighter_name = "Cory Sandhagen"

# Load the dataset
master_df = pd.read_csv('~/Deployed/mma-ai/Scrapers/data/github/master.csv')

# Filter the DataFrame to get all fights of the given fighter in chronological order
fighter_fights = master_df[
    master_df['FIGHTER1'].str.contains(fighter_name, case=False, na=False) | 
    master_df['FIGHTER2'].str.contains(fighter_name, case=False, na=False)
].sort_values(by='DATE')

# Check if there are any fights for the given fighter
if fighter_fights.empty:
    print(f"No fights found for {fighter_name}")
else:
    # Extract detailed information
    fighter_details = fighter_fights[['EVENT', 'BOUT', 'OUTCOME', 'WEIGHTCLASS', 'METHOD', 'ROUND', 'TIME', 'TIME FORMAT', 'REFEREE', 'DETAILS', 'URL', 'DATE', 'LOCATION', 'FIGHTER1', 'FIGHTER2']].copy()
    
    # Extract opponents and outcomes
    opponents_and_outcomes = fighter_fights[['DATE', 'FIGHTER1', 'FIGHTER2', 'OUTCOME', 'WINNING_FIGHTER']].copy()
    opponents_and_outcomes['OPPONENT'] = opponents_and_outcomes.apply(
        lambda row: row['FIGHTER2'] if fighter_name.lower() in row['FIGHTER1'].lower() else row['FIGHTER1'], axis=1
    )
    opponents_and_outcomes['RESULT'] = opponents_and_outcomes.apply(
        lambda row: 'Win' if (fighter_name.lower() in row['WINNING_FIGHTER'].lower()) else 'Loss', axis=1
    )
    
    # Combine details with opponents and outcomes
    detailed_fights = pd.concat([opponents_and_outcomes, fighter_details.drop(columns=['DATE', 'FIGHTER1', 'FIGHTER2', 'OUTCOME'])], axis=1)
    
    # Display the detailed fights
    print(detailed_fights)


            DATE           FIGHTER1        FIGHTER2 OUTCOME  \
2623  2018-01-27      austin arnett  cory sandhagen     L/W   
2406  2018-08-25     cory sandhagen  iuri alcantara     W/L   
2257  2019-01-19     cory sandhagen  mario bautista     W/L   
2129  2019-04-27       john lineker  cory sandhagen     L/W   
1998  2019-08-17   raphael assuncao  cory sandhagen     L/W   
1714  2020-06-06  aljamain sterling  cory sandhagen     W/L   
1550  2020-10-10      marlon moraes  cory sandhagen     L/W   
1425  2021-02-06     cory sandhagen   frankie edgar     W/L   
1223  2021-07-24     cory sandhagen    tj dillashaw     L/W   
1111  2021-10-30           petr yan  cory sandhagen     W/L   
727   2022-09-17     cory sandhagen     song yadong     W/L   
519   2023-03-25        marlon vera  cory sandhagen     L/W   
349   2023-08-05     cory sandhagen        rob font     W/L   

        WINNING_FIGHTER           OPPONENT RESULT  \
2623     Cory Sandhagen      austin arnett    Win   
2406     Co

In [11]:
import pandas as pd

# Load the dataset
master_df = pd.read_csv('~/Deployed/mma-ai/Scrapers/data/github/master.csv')

# Function to get champions and their reigns for a specific weight class using the WEIGHTCLASS column
def get_champions_and_reigns(weight_class):
    # Filter the DataFrame for the specified weight class title bouts
    title_fights = master_df[
        master_df['WEIGHTCLASS'].str.contains(weight_class, case=False, na=False)
    ].sort_values(by='DATE')
    
    # Extract relevant information
    champions_opponents_dates = title_fights[['DATE', 'EVENT', 'BOUT', 'WINNING_FIGHTER', 'FIGHTER1', 'FIGHTER2']].copy()
    champions_opponents_dates = champions_opponents_dates.sort_values(by='DATE')
    
    # Initialize the list to store champions and reigns
    champions_list = []
    current_champion = None
    current_start_date = None
    
    for index, row in champions_opponents_dates.iterrows():
        opponent = row['FIGHTER2'] if row['WINNING_FIGHTER'] == row['FIGHTER1'] else row['FIGHTER1']
        if current_champion is None:
            current_champion = row['WINNING_FIGHTER']
            current_start_date = row['DATE']
        elif current_champion != row['WINNING_FIGHTER']:
            champions_list.append(f"{current_champion} - {current_start_date} to {row['DATE']} (Defeated {opponent})")
            current_champion = row['WINNING_FIGHTER']
            current_start_date = row['DATE']
    
    if current_champion is not None:
        champions_list.append(f"{current_champion} - {current_start_date} to Present")
    
    return champions_list

# Define all weight classes as per the dataset
weight_classes = [
    'UFC Featherweight Title Bout', 'UFC Interim Featherweight Title Bout', 'UFC Lightweight Title Bout', 'UFC Interim Lightweight Title Bout',
    'UFC Welterweight Title Bout', 'UFC Interim Welterweight Title Bout', 'UFC Middleweight Title Bout', 'UFC Interim Middleweight Title Bout',
    'UFC Light Heavyweight Title Bout', 'UFC Interim Light Heavyweight Title Bout', 'UFC Heavyweight Title Bout', 'UFC Interim Heavyweight Title Bout',
    'UFC Flyweight Title Bout', 'UFC Interim Flyweight Title Bout', 'UFC Bantamweight Title Bout', 'UFC Interim Bantamweight Title Bout'
]

# Get champions and their reigns for all weight classes
all_champions = {}
for wc in weight_classes:
    champions = get_champions_and_reigns(wc)
    all_champions[wc] = champions

# Print the champions and their reigns for all weight classes
champions_output = ""
for wc, champions in all_champions.items():
    champions_output += f"\n{wc} Champions and Reigns:\n"
    if champions:
        for champ in champions:
            champions_output += champ + "\n"
    else:
        champions_output += "No data available for this weight class.\n"

print(champions_output)



UFC Featherweight Title Bout Champions and Reigns:
Jose Aldo - 2011-04-30 to 2015-12-12 (Defeated jose aldo)
Conor McGregor - 2015-12-12 to 2017-06-03 (Defeated jose aldo)
Max Holloway - 2017-06-03 to 2019-12-14 (Defeated max holloway)
Alexander Volkanovski - 2019-12-14 to 2024-02-17 (Defeated alexander volkanovski)
Ilia Topuria - 2024-02-17 to Present

UFC Interim Featherweight Title Bout Champions and Reigns:
Conor McGregor - 2015-07-11 to 2016-07-09 (Defeated jose aldo)
Jose Aldo - 2016-07-09 to 2016-12-10 (Defeated max holloway)
Max Holloway - 2016-12-10 to 2023-02-11 (Defeated yair rodriguez)
Yair Rodriguez - 2023-02-11 to Present

UFC Lightweight Title Bout Champions and Reigns:
Sean Sherk - 2006-10-14 to 2008-01-19 (Defeated bj penn)
BJ Penn - 2008-01-19 to 2010-04-10 (Defeated bj penn)
Frankie Edgar - 2010-04-10 to 2011-01-01 (Defeated frankie edgar)
Draw - 2011-01-01 to 2011-10-08 (Defeated frankie edgar)
Frankie Edgar - 2011-10-08 to 2012-02-25 (Defeated frankie edgar)
Benso